In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd; import sys
sys.path.append('..')
import numpy as np
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = 100
from IPython.core.display import display, HTML
import re
import ast
display(HTML("<style>.container { width:100% !important; }</style>"))
from tqdm import tqdm 
import traceback
import imp

#### EXPLAIN ANALYZE

Оценка перфоманса sql запроса с декомпозицией по шагам  
Рассмотрено на примере presto - так как там действует mapReduce схема, то  
есть ряд операторов которые в том числе распараллеливают вычисления и.т.д  
Список возможных операторов приведен ниже, новые также можно гуглить

In [1]:
# пример запроса выгруженного из trino
query_plan = """Trino version: 409-dirty\nQueued: 103.92us, Analysis: 172.36ms, Planning: 52.39ms, Execution: 3.34s\nFragment 1 [SINGLE]\n    CPU: 150.37ms, Scheduled: 625.24ms, Blocked 3.07m (Input: 1.50m, Output: 0.00ns), Input: 3040 rows (53.44kB); per task: avg.: 3040.00 std.dev.: 0.00, Output: 1 row (23B)\n    Output layout: [count_6, expr_7, count]\n    Output partitioning: SINGLE []\n    Project[]\n    │   Layout: [count_6:bigint, expr_7:integer, count:bigint]\n    │   Estimates: {rows: 1 (23B), cpu: 23, memory: 0B, network: 0B}\n    │   CPU: 1.00ms (0.01%), Scheduled: 1.00ms (0.00%), Blocked: 0.00ns (0.00%), Output: 1 row (23B)\n    │   Input avg.: 0.03 rows, Input std.dev.: 556.78%\n    │   expr_7 := 2\n    └─ LocalExchange[partitioning = ROUND_ROBIN]\n       │   Layout: [count_6:bigint, count:bigint]\n       │   Estimates: {rows: 1 (18B), cpu: 18, memory: 0B, network: 0B}\n       │   CPU: 0.00ns (0.00%), Scheduled: 0.00ns (0.00%), Blocked: 48.45s (0.19%), Output: 1 row (18B)\n       │   Input avg.: 1.00 rows, Input std.dev.: 0.00%\n       └─ Aggregate[type = FINAL]\n          │   Layout: [count_6:bigint, count:bigint]\n          │   Estimates: {rows: 1 (18B), cpu: ?, memory: 18B, network: 0B}\n          │   CPU: 17.00ms (0.12%), Scheduled: 17.00ms (0.03%), Blocked: 0.00ns (0.00%), Output: 1 row (18B)\n          │   Input avg.: 3040.00 rows, Input std.dev.: 0.00%\n          │   count_6 := count("count_8")\n          │   count := count("count_9")\n          └─ LocalExchange[partitioning = SINGLE]\n             │   Layout: [count_9:bigint, count_8:bigint]\n             │   Estimates: {rows: ? (?), cpu: 0, memory: 0B, network: 0B}\n             │   CPU: 23.00ms (0.16%), Scheduled: 22.00ms (0.03%), Blocked: 2.80s (0.01%), Output: 3040 rows (53.44kB)\n             │   Input avg.: 95.00 rows, Input std.dev.: 32.16%\n             └─ RemoteSource[sourceFragmentIds = [2]]\n                    Layout: [count_9:bigint, count_8:bigint]\n                    CPU: 91.00ms (0.64%), Scheduled: 562.00ms (0.84%), Blocked: 1.50m (0.35%), Output: 3040 rows (53.44kB)\n                    Input avg.: 95.00 rows, Input std.dev.: 32.16%\n\nFragment 2 [HASH]\n    CPU: 1.85s, Scheduled: 13.04s, Blocked 3.53h (Input: 1.73h, Output: 0.00ns), Input: 263673 rows (4.76MB); per task: avg.: 2775.51 std.dev.: 11600.81, Output: 3040 rows (53.44kB)\n    Amount of input data processed by the workers for this stage might be skewed\n    Output layout: [count_9, count_8]\n    Output partitioning: SINGLE []\n    Aggregate[type = PARTIAL]\n    │   Layout: [count_9:bigint, count_8:bigint]\n    │   CPU: 397.00ms (2.78%), Scheduled: 1.41s (2.10%), Blocked: 0.00ns (0.00%), Output: 3040 rows (53.44kB)\n    │   Input avg.: 86.73 rows, Input std.dev.: 2377.07%\n    │   count_9 := count("employer_id") (mask = employer_id$distinct)\n    │   count_8 := count("expr") (mask = expr$distinct)\n    └─ MarkDistinct[distinct = [expr:integer], marker = expr$distinct, hash = [$hashvalue]]\n       │   Layout: [expr:integer, employer_id:integer, employer_id$distinct:boolean, $hashvalue:bigint, expr$distinct:boolean]\n       │   CPU: 193.00ms (1.35%), Scheduled: 1.07s (1.59%), Blocked: 0.00ns (0.00%), Output: 263673 rows (4.77MB)\n       │   Input avg.: 86.73 rows, Input std.dev.: 2377.07%\n       └─ LocalExchange[partitioning = HASH, hashColumn = [$hashvalue], arguments = ["expr"]]\n          │   Layout: [expr:integer, employer_id:integer, employer_id$distinct:boolean, $hashvalue:bigint]\n          │   Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n          │   CPU: 230.00ms (1.61%), Scheduled: 3.99s (5.93%), Blocked: 1.57h (22.20%), Output: 263673 rows (4.76MB)\n          │   Input avg.: 86.73 rows, Input std.dev.: 440.62%\n          └─ RemoteSource[sourceFragmentIds = [3]]\n                 Layout: [expr:integer, employer_id:integer, employer_id$distinct:boolean, $hashvalue_10:bigint]\n                 CPU: 290.00ms (2.03%), Scheduled: 1.85s (2.76%), Blocked: 1.73h (24.47%), Output: 263673 rows (4.76MB)\n                 Input avg.: 86.73 rows, Input std.dev.: 440.62%\n\nFragment 3 [HASH]\n    CPU: 9.86s, Scheduled: 48.14s, Blocked 2.17h (Input: 1.08h, Output: 0.00ns), Input: 263673 rows (7.04MB); per task: avg.: 2775.51 std.dev.: 35.88, Output: 263673 rows (4.76MB)\n    Output layout: [expr, employer_id, employer_id$distinct, $hashvalue_11]\n    Output partitioning: HASH [expr][$hashvalue_11]\n    Project[]\n    │   Layout: [expr:integer, employer_id:integer, $hashvalue_11:bigint, employer_id$distinct:boolean]\n    │   Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n    │   CPU: 3.50s (24.54%), Scheduled: 8.34s (12.42%), Blocked: 0.00ns (0.00%), Output: 263673 rows (4.76MB)\n    │   Input avg.: 86.73 rows, Input std.dev.: 10.50%\n    └─ MarkDistinct[distinct = [employer_id:integer], marker = employer_id$distinct, hash = [$hashvalue_12]]\n       │   Layout: [expr:integer, employer_id:integer, $hashvalue_11:bigint, $hashvalue_12:bigint, employer_id$distinct:boolean]\n       │   CPU: 2.05s (14.36%), Scheduled: 4.72s (7.03%), Blocked: 0.00ns (0.00%), Output: 263673 rows (7.31MB)\n       │   Input avg.: 86.73 rows, Input std.dev.: 10.50%\n       └─ LocalExchange[partitioning = HASH, hashColumn = [$hashvalue_12], arguments = ["employer_id"]]\n          │   Layout: [expr:integer, employer_id:integer, $hashvalue_11:bigint, $hashvalue_12:bigint]\n          │   Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n          │   CPU: 2.09s (14.66%), Scheduled: 6.80s (10.12%), Blocked: 1.04h (14.69%), Output: 263673 rows (7.04MB)\n          │   Input avg.: 86.73 rows, Input std.dev.: 152.26%\n          └─ RemoteSource[sourceFragmentIds = [4]]\n                 Layout: [expr:integer, employer_id:integer, $hashvalue_13:bigint, $hashvalue_14:bigint]\n                 CPU: 1.11s (7.80%), Scheduled: 24.95s (37.14%), Blocked: 1.08h (15.29%), Output: 263673 rows (7.04MB)\n                 Input avg.: 86.73 rows, Input std.dev.: 152.26%\n\nFragment 4 [HASH]\n    CPU: 2.87s, Scheduled: 14.17s, Blocked 1.80h (Input: 59.47m, Output: 0.00ns), Input: 327832 rows (7.42MB); per task: avg.: 3450.86 std.dev.: 7621.27, Output: 263673 rows (7.04MB)\n    Amount of input data processed by the workers for this stage might be skewed\n    Output layout: [expr, employer_id, $hashvalue_21, $hashvalue_20]\n    Output partitioning: HASH [employer_id][$hashvalue_20]\n    Project[]\n    │   Layout: [expr:integer, employer_id:integer, $hashvalue_20:bigint, $hashvalue_21:bigint]\n    │   Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n    │   CPU: 810.00ms (5.68%), Scheduled: 5.51s (8.20%), Blocked: 0.00ns (0.00%), Output: 263673 rows (7.04MB)\n    │   Input avg.: 86.73 rows, Input std.dev.: 638.70%\n    │   $hashvalue_21 := combine_hash(bigint \'0\', COALESCE("$operator$hash_code"("expr"), 0))\n    └─ Project[]\n       │   Layout: [expr:integer, employer_id:integer, $hashvalue_20:bigint]\n       │   Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n       │   CPU: 314.00ms (2.20%), Scheduled: 593.00ms (0.88%), Blocked: 0.00ns (0.00%), Output: 263673 rows (5.90MB)\n       │   Input avg.: 86.73 rows, Input std.dev.: 638.70%\n       │   expr := (CASE WHEN ("discard_status" = 0) THEN "employer_id" END)\n       │   $hashvalue_20 := combine_hash(bigint \'0\', COALESCE("$operator$hash_code"("employer_id"), 0))\n       └─ InnerJoin[criteria = ("area_id" = "area_id_1"), hash = [$hashvalue_15, $hashvalue_17], distribution = PARTITIONED]\n          │   Layout: [employer_id:integer, discard_status:integer]\n          │   Estimates: {rows: ? (?), cpu: ?, memory: ?, network: 0B}\n          │   CPU: 829.00ms (5.81%), Scheduled: 2.84s (4.23%), Blocked: 22.76m (5.37%), Output: 263673 rows (4.40MB)\n          │   Left (probe) Input avg.: 104.95 rows, Input std.dev.: 555.73%\n          │   Right (build) Input avg.: 2.89 rows, Input std.dev.: 57.45%\n          │   Distribution: PARTITIONED\n          │   dynamicFilterAssignments = {area_id_1 -> #df_743}\n          ├─ RemoteSource[sourceFragmentIds = [5]]\n          │      Layout: [employer_id:integer, area_id:integer, discard_status:integer, $hashvalue_15:bigint]\n          │      CPU: 3.00ms (0.02%), Scheduled: 19.00ms (0.03%), Blocked: 45.15m (10.64%), Output: 319057 rows (7.30MB)\n          │      Input avg.: 104.95 rows, Input std.dev.: 555.73%\n          └─ LocalExchange[partitioning = HASH, hashColumn = [$hashvalue_17], arguments = ["area_id_1"]]\n             │   Layout: [area_id_1:integer, $hashvalue_17:bigint]\n             │   Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n             │   CPU: 108.00ms (0.76%), Scheduled: 513.00ms (0.76%), Blocked: 14.45m (3.41%), Output: 8775 rows (119.97kB)\n             │   Input avg.: 2.89 rows, Input std.dev.: 558.74%\n             └─ RemoteSource[sourceFragmentIds = [6]]\n                    Layout: [area_id_1:integer, $hashvalue_18:bigint]\n                    CPU: 0.00ns (0.00%), Scheduled: 28.00ms (0.04%), Blocked: 14.32m (3.38%), Output: 8775 rows (119.97kB)\n                    Input avg.: 2.89 rows, Input std.dev.: 558.74%\n\nFragment 5 [SOURCE]\n    CPU: 2.20s, Scheduled: 3.91s, Blocked 0.00ns (Input: 0.00ns, Output: 0.00ns), Input: 10789427 rows (239.22MB); per task: avg.: 1078942.70 std.dev.: 720610.15, Output: 319057 rows (7.30MB)\n    Output layout: [employer_id, area_id, discard_status, $hashvalue_16]\n    Output partitioning: HASH [area_id][$hashvalue_16]\n    ScanFilterProject[table = hive:snapshot2:employer, filterPredicate = ("creation_time" > TIMESTAMP \'2025-01-01 00:00:00.000000\'), dynamicFilters = {"area_id" = #df_743}]\n        Layout: [employer_id:integer, area_id:integer, discard_status:integer, $hashvalue_16:bigint]\n        Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}/{rows: ? (?), cpu: ?, memory: 0B, network: 0B}/{rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n        CPU: 2.20s (15.41%), Scheduled: 3.91s (5.82%), Blocked: 0.00ns (0.00%), Output: 319057 rows (7.30MB)\n        Input avg.: 719295.13 rows, Input std.dev.: 95.74%\n        $hashvalue_16 := combine_hash(bigint \'0\', COALESCE("$operator$hash_code"("area_id"), 0))\n        creation_time := creation_time:timestamp:REGULAR\n        area_id := area_id:int:REGULAR\n        discard_status := discard_status:int:REGULAR\n        employer_id := employer_id:int:REGULAR\n        Input: 10789427 rows (239.22MB), Filtered: 97.04%, Physical input: 99.30MB, Physical input time: 1062.00ms\n        Dynamic filters: \n            - df_743, [ SortedRangeSet[type=integer, ranges=8775, {[1], ..., [11876]}] ], collection time=1.11s\n\nFragment 6 [SOURCE]\n    CPU: 9.15ms, Scheduled: 26.39ms, Blocked 0.00ns (Input: 0.00ns, Output: 0.00ns), Input: 10157 rows (93.81kB); per task: avg.: 10157.00 std.dev.: 0.00, Output: 8775 rows (119.97kB)\n    Output layout: [area_id_1, $hashvalue_19]\n    Output partitioning: HASH [area_id_1][$hashvalue_19]\n    ScanFilterProject[table = hive:snapshot2:area, filterPredicate = ("country_name" = VARCHAR U&\'\\0420\\043E\\0441\\0441\\0438\\044F\')]\n        Layout: [area_id_1:integer, $hashvalue_19:bigint]\n        Estimates: {rows: ? (?), cpu: ?, memory: 0B, network: 0B}/{rows: ? (?), cpu: ?, memory: 0B, network: 0B}/{rows: ? (?), cpu: ?, memory: 0B, network: 0B}\n        CPU: 9.00ms (0.06%), Scheduled: 27.00ms (0.04%), Blocked: 0.00ns (0.00%), Output: 8775 rows (119.97kB)\n        Input avg.: 10157.00 rows, Input std.dev.: 0.00%\n        $hashvalue_19 := combine_hash(bigint \'0\', COALESCE("$operator$hash_code"("area_id_1"), 0))\n        area_id_1 := area_id:int:REGULAR\n        country_name := country_name:string:REGULAR\n        Input: 10157 rows (93.81kB), Filtered: 13.61%, Physical input: 269.29kB, Physical input time: 5950000.00ns\n\n"""

In [3]:
# ФУНКЦИИ ДЛЯ ПАРСИНГА
def get_cpu_of_fragments(query_plan):
    def to_seconds(s):
        if 'ms' in s or 'us' in s:
            return 0        
        num = float(s[:-1])
        unit = s[-1]
        return int(num * 3600 if unit == 'h' else num * 60)
    parse = query_plan.split('Fragment ')
    frag_num_list = []
    frag_list = []
    cpu_list = []
    cpu_in_sec_list = []
    for idx, frag in enumerate(parse):
        if idx == 0:
            continue
        cpu = frag.split('CPU: ')[1].split('Input')[0][:-2]
        frag_num_list.append(idx)
        cpu_list.append(cpu)
        cpu_in_sec_list.append(to_seconds(cpu.split(', Scheduled: ')[0]))
        frag_list.append(('Fragment ' + frag).strip())

    return pd.DataFrame({'frag_num' : frag_num_list,
                         'CPU & Scheduled' : cpu_list,
                         'CPU_in_sec' : cpu_in_sec_list, 'frag' : frag_list}).sort_values(by='CPU_in_sec', ascending=False)

def remove_combine_hash(text):
    pattern = re.compile(r'combine_hash\(')
    while True:
        start = text.find('combine_hash(')
        if start == -1:
            break
        # найти соответствующую закрывающую скобку
        count = 0
        for i in range(start, len(text)):
            if text[i] == '(':
                count += 1
            elif text[i] == ')':
                count -= 1
                if count == 0:
                    end = i
                    break
        # simplifier combine_hash(...)
        text = text[:start] + 'combine_hash[...]' + text[end+1:]
    return text

def get_frag(df_frag, frag_num, simplify = True):
    def get_fat(frag, expr):
        return frag.replace(expr, f"\033[4m{expr}\033[0m""")
    # получаем frag
    frag = df_frag[df_frag.frag_num == frag_num].frag.iloc[0]
    if simplify:
        cleaned = re.sub(r'Layout:\s*\[.*?\]', 'Layout:[...]', frag)
        cleaned = re.sub(r'Output layout:\s*\[.*?\]', 'Output Layout:[...]', cleaned)
        cleaned = re.sub(r'\n.*Estimates.*\n', "\n", cleaned)
        cleaned = re.sub(r'\n.*Input avg.:.*\n', "\n", cleaned)
        frag = remove_combine_hash(cleaned)
    # frag = re.sub(r'(CPU:\s*\d+\.\d+\w)', r'\033[1;4m\1\033[0m', frag)
    frag = re.sub(r'(CPU:\s*\d+\.\d+\w(?: \(\d+\.\d+%\))?)', r'\033[1;4m\1\033[0m', frag)
    expr_list = ['Aggregate', 'ScanFilterProject', 'InnerJoin', 'LeftJoin',
                 'RemoteSource', 'LocalExchange', 'FilterProject', 'Project', 'Window', 'ScanProject', 'RemoteExchange', 'MarkDistinct']
    for expr in expr_list:
        frag = get_fat(frag, expr)
    
    print(frag)

def get_total_perfomance(query_plan):
    print(query_plan.split('Fragment')[0])

In [7]:
# перфоманс всего запроса
get_total_perfomance(query_plan)

Trino version: 409-dirty
Queued: 103.92us, Analysis: 172.36ms, Planning: 52.39ms, Execution: 3.34s



In [9]:
# получаем список Fragment с CPU, отсортированных по тяжести
df = get_cpu_of_fragments(query_plan)
df

,frag_num,CPU & Scheduled,CPU_in_sec,frag
2,3,"9.86s, Scheduled: 48.14s, Blocked 2.17h",591,"Fragment 3 [HASH]\n CPU: 9.86s, Scheduled: 48.14s, Blocked 2.17h (Input: 1.08h, Output: 0.00n..."
3,4,"2.87s, Scheduled: 14.17s, Blocked 1.80h",172,"Fragment 4 [HASH]\n CPU: 2.87s, Scheduled: 14.17s, Blocked 1.80h (Input: 59.47m, Output: 0.00..."
4,5,"2.20s, Scheduled: 3.91s, Blocked 0.00ns",132,"Fragment 5 [SOURCE]\n CPU: 2.20s, Scheduled: 3.91s, Blocked 0.00ns (Input: 0.00ns, Output: 0...."
1,2,"1.85s, Scheduled: 13.04s, Blocked 3.53h",111,"Fragment 2 [HASH]\n CPU: 1.85s, Scheduled: 13.04s, Blocked 3.53h (Input: 1.73h, Output: 0.00n..."
0,1,"150.37ms, Scheduled: 625.24ms, Blocked 3.07m",0,"Fragment 1 [SINGLE]\n CPU: 150.37ms, Scheduled: 625.24ms, Blocked 3.07m (Input: 1.50m, Output..."
5,6,"9.15ms, Scheduled: 26.39ms, Blocked 0.00ns",0,"Fragment 6 [SOURCE]\n CPU: 9.15ms, Scheduled: 26.39ms, Blocked 0.00ns (Input: 0.00ns, Output:..."


In [10]:
# simplify - свернуть в фрагменте большие куски метаинформации (layout с набором колонок , combinehash итд)
#  Fragment разбивается на дерево операторов которые работают с данными и занимают cpu - его значения и процент от тотал cpu подчеркнуты
# также подчеркнуты названия операторов, список их значений ниже (можно пополнять)

# при оценке перфоманса
# CPU: 16.56s, Scheduled: 1.18m, Blocked 0.00ns
# cpu_time = полное время выполнения запроса на всех ядрах (если их много то время превышает реальное время выполнения запроса из за параллельности)
# scheduled = полное время ожидания ресурса cpu на всех ядрах (зависит от других запросов в кластере итд - шумная штука)
# blocked = полное время ожидания из за блокировок/чтения-записи с диска/доступа к буферам итд
# оптимизировать свой запрос оптимальнее всего следя за тем как снижается cpu_time

get_frag(df, frag_num = 5, simplify=True)

Fragment 5 [SOURCE]
    CPU: 2.20s, Scheduled: 3.91s, Blocked 0.00ns (Input: 0.00ns, Output: 0.00ns), Input: 10789427 rows (239.22MB); per task: avg.: 1078942.70 std.dev.: 720610.15, Output: 319057 rows (7.30MB)
    Output Layout:[...]
    Output partitioning: HASH [area_id][$hashvalue_16]
    ScanFilterProject[table = hive:snapshot2:employer, filterPredicate = ("creation_time" > TIMESTAMP '2025-01-01 00:00:00.000000'), dynamicFilters = {"area_id" = #df_743}]
        Layout:[...]
        CPU: 2.20s (15.41%), Scheduled: 3.91s (5.82%), Blocked: 0.00ns (0.00%), Output: 319057 rows (7.30MB)
        $hashvalue_16 := combine_hash[...]
        creation_time := creation_time:timestamp:REGULAR
        area_id := area_id:int:REGULAR
        discard_status := discard_status:int:REGULAR
        employer_id := employer_id:int:REGULAR
        Input: 10789427 rows (239.22MB), Filtered: 97.04%, Physical input: 99.30MB, Physical input time: 1062.00ms
        Dynamic filters: 
            - df_743, [ So

In [ ]:
# разные виды операторов, список можно дополнять

'Aggregate' # агрегация с совершением операций вроде max, sum ...

'InnerJoin' #  джойны
'LeftJoin'

'MarkDistinct' # помечает уникальные строки (для операций типа count distinct ...)

'RemoteSource'  # получение данных из другого узла (например заливаем таблицу на ноду для ее обработки)

'LocalExchange' # перегрупировка данных перед совершением операции на одной local ноде
'RemoteExchange' # перераспределение данных между нодами для параллельного выполнения операции


'Window' # вычисление оконки

'FilterProject' # фильтрация по типу where итд
'Project' # оператор типа calc_field когда надо получить колонку из др колонок (например Y = X * 2) - проекция
'ScanProject' # сканирует таблицу + выбирает нужные колонки (проекция)
'ScanFilterProject' # отбор данных вместе с фильтрацией